In [3]:
import pandas as pd
import os
import numpy as np

# Grabbing Features

In [18]:
def build_data(file_names,directory):
    df_Final = pd.DataFrame(columns=['PDB code','VDWAALS','EEL','EGB','ESURF','GGAS','GSOLV','TOTAL'])
    for file in file_names:
        df = pd.read_csv(directory+"/"+file,delimiter='\t')
        df = df.iloc[83:92, :]
        df.drop([85,86],axis = 0,inplace=True)
        l1 = [file[:4]]
        for _,row in df.iterrows():
            val = float(row.iloc[0].split()[1])
            l1.append(val)
        df_Final.loc[len(df_Final.index)] = l1
    return df_Final

def create_pairs(df):
    df_Final = pd.DataFrame(columns=["PDB code 1","VDWAALS 1","EEL 1","EGB 1","ESURF 1","GGAS 1","GSOLV 1","TOTAL 1","PDB code 2","VDWAALS 2","EEL 2","EGB 2","ESURF 2","GGAS 2","GSOLV 2","TOTAL 2"])
    for _ , row1 in df.iterrows():
        for _ , row2 in df.iterrows():
            if row1["PDB code"] == row2["PDB code"]:
                continue
            temp = row1.to_list() + row2.to_list()
            df_Final.loc[len(df_Final.index)] = temp
    return df_Final

In [19]:
directory = "complex_data_raw"
df_features = build_data(os.listdir(directory),directory)
df_features = create_pairs(df_features)

In [20]:
df_features

,PDB code 1,VDWAALS 1,EEL 1,EGB 1,ESURF 1,GGAS 1,GSOLV 1,TOTAL 1,PDB code 2,VDWAALS 2,EEL 2,EGB 2,ESURF 2,GGAS 2,GSOLV 2,TOTAL 2
0,4him,-114.69,-645.74,676.69,-13.70,-760.43,663.00,-97.43,1u63,-279.25,-34181.81,34208.24,-41.36,-34461.07,34166.88,-294.19
1,4him,-114.69,-645.74,676.69,-13.70,-760.43,663.00,-97.43,2kh9,-49.13,-508.76,534.32,-5.94,-557.89,528.38,-29.51
2,4him,-114.69,-645.74,676.69,-13.70,-760.43,663.00,-97.43,1wwe,-89.90,-1356.54,1369.99,-11.95,-1446.44,1358.04,-88.40
3,4him,-114.69,-645.74,676.69,-13.70,-760.43,663.00,-97.43,1osb,-563.32,-11193.39,11154.77,-72.18,-11756.71,11082.59,-674.11
4,4him,-114.69,-645.74,676.69,-13.70,-760.43,663.00,-97.43,5yts,-57.48,-136.57,171.71,-7.00,-194.05,164.71,-29.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,1rkj,-119.28,-801.67,834.16,-15.17,-920.95,818.99,-101.96,2lec,-113.05,-619.56,647.62,-13.80,-732.61,633.82,-98.79
1802,1rkj,-119.28,-801.67,834.16,-15.17,-920.95,818.99,-101.96,5wwe,-78.63,-1194.77,1191.48,-10.79,-1273.40,1180.70,-92.70
1803,1rkj,-119.28,-801.67,834.16,-15.17,-920.95,818.99,-101.96,1wwf,-109.69,-1452.29,1473.77,-14.75,-1561.98,1459.02,-102.95
1804,1rkj,-119.28,-801.67,834.16,-15.17,-920.95,818.99,-101.96,2mki,-85.56,-260.76,305.67,-10.57,-346.32,295.11,-51.21


# Data outputs

In [22]:
def translate_kd(kd):
    kd.strip()
    kd = kd[3:]
    factor = 0
    if "nM" in kd:
        factor = 1
    elif "pM" in kd:
        factor = .001
    elif "uM" in kd:
        factor = 1000
    kd = kd[:-2]
    try:
        return float(kd) * factor
    except:
        print(kd)
        return None
    
def output(df):
    df_Final = pd.DataFrame(columns=["PDB code 1","Protein Name 1","Ligand Name 1","Affinity Data 1","kd value (nM) 1","PDB code 2","Protein Name 2","Ligand Name 2","Affinity Data 2","kd value (nM) 2","Output"])
    for _ , row1 in df.iterrows():
        for _ , row2 in df.iterrows():
            if row1["PDB code"] == row2["PDB code"]:
                continue
            output = 1 if row1["kd value (nM)"] < row2["kd value (nM)"] else 0
            temp = row1.to_list() + row2.to_list()
            temp.append(output)
            df_Final.loc[len(df_Final.index)] = temp
    return df_Final

In [23]:
df = pd.read_csv("KD_data.csv")
df.columns = df.iloc[0]
df.drop(index = 0,inplace=True)
df = df[['PDB code','Protein Name','Ligand Name','Affinity Data']]
df["kd value (nM)"] = [translate_kd(kd) for kd in df["Affinity Data"]]
df = df[df["PDB code"].isin(df_features["PDB code 1"])]
df_outputs = output(df)

0=4.9
0=0.03
0=0.3
0=4.6
0=4
0=0.25
0=126


In [25]:
df_outputs

,PDB code 1,Protein Name 1,Ligand Name 1,Affinity Data 1,kd value (nM) 1,PDB code 2,Protein Name 2,Ligand Name 2,Affinity Data 2,kd value (nM) 2,Output
0,1exd,GLUTAMINYL-TRNA SYNTHETASE,GLUTAMINE TRNA APTAMER,Kd=7.1nM,7.1,1il2,ASPARTYL-TRNA SYNTHETASE,ASPARTYL TRANSFER RNA,Kd=3uM,3000.00,1
1,1exd,GLUTAMINYL-TRNA SYNTHETASE,GLUTAMINE TRNA APTAMER,Kd=7.1nM,7.1,1je8,Nitrate/Nitrite Response Regulator Protein NARL,DNA,Kd=0.15nM,0.15,0
2,1exd,GLUTAMINYL-TRNA SYNTHETASE,GLUTAMINE TRNA APTAMER,Kd=7.1nM,7.1,1omh,trwC protein,DNA OLIGONUCLEOTIDE,Kd=70nM,70.00,1
3,1exd,GLUTAMINYL-TRNA SYNTHETASE,GLUTAMINE TRNA APTAMER,Kd=7.1nM,7.1,1osb,trwC protein,DNA OLIGONUCLEOTIDE,Kd=70nM,70.00,1
4,1exd,GLUTAMINYL-TRNA SYNTHETASE,GLUTAMINE TRNA APTAMER,Kd=7.1nM,7.1,1rkj,Nucleolin,RNA,Kd=1.1uM,1100.00,1
...,...,...,...,...,...,...,...,...,...,...,...
1801,5yts,"Nuclease-sensitive element-binding protein 1, ...",RNA A2U (UCUUCU),Kd=2.77uM,2770.0,5wzk,Pumilio homolog 23 (APUM23 85-655) deletion-of...,GGAAUUGACGG RNA,Kd=0.16uM,160.00,0
1802,5yts,"Nuclease-sensitive element-binding protein 1, ...",RNA A2U (UCUUCU),Kd=2.77uM,2770.0,5z6z,"human DUX4 homeodomains, DUX4-DH",DNA1,Kd=45.24nM,45.24,0
1803,5yts,"Nuclease-sensitive element-binding protein 1, ...",RNA A2U (UCUUCU),Kd=2.77uM,2770.0,5ytx,"Nuclease-sensitive element-binding protein 1, ...",RNA U3A (UCAACU),Kd=1.34uM,1340.00,0
1804,5yts,"Nuclease-sensitive element-binding protein 1, ...",RNA A2U (UCUUCU),Kd=2.77uM,2770.0,5ytv,"Nuclease-sensitive element-binding protein 1, ...",RNA CAUC(UCAUCU),Kd=1.26uM,1260.00,0


# More features

In [9]:
def read_file1(complex,file):
    try:
        _,y = np.loadtxt("hbond_analysis/" + complex+'/'+file,comments = ['@','#'],unpack=True)
        return np.mean(y), np.std(y)
    except:
        return None,None

def read_file2(complex,file):
    try:
        _,y,_ = np.loadtxt("hbond_analysis/" + complex+'/'+file,comments = ['@','#'],unpack=True)
        return np.mean(y), np.std(y)
    except:
        return None,None

def read_file3(complex,file):
    try:
        _,y,z = np.loadtxt("hbond_analysis/" + complex+'/'+file,comments = ['@','#'],unpack=True)
        return np.mean(y), np.std(y), np.mean(z), np.mean(y)
    except:
        return None,None

In [12]:

l1 = os.listdir("hbond_analysis")
#df_extra_features = pd.DataFrame(columns=['PDB code','hbdist_mean','hbdist_std','hblife_mean','hblife_std','hbnum_mean1','hbnum_std1','hbnum_mean2','hbnum_std2'])
df_extra_features = pd.DataFrame(columns=['PDB code','hbdist_mean','hbdist_std','hbnum_mean1','hbnum_std1','hbnum_mean2','hbnum_std2'])
for complex in l1:
    l2 = os.listdir("hbond_analysis/" + complex)
    l1 = [complex]
    for file in l2:
        if file == 'hbdist.xvg':
            mean, std = read_file1(complex,file)
            l1 += [mean,std]
        elif file == 'hblife.xvg':
            #mean, std = read_file2(complex,file)
            #l1 += [mean,std]
            continue
        elif file == 'hbnum.xvg':
            mean1, std1, mean2, std2 = read_file3(complex,file)
            l1 += [mean1,std1,mean2,std2]
    
    df_extra_features.loc[len(df.index)] = l1

In [13]:
df_extra_features

,PDB code,hbdist_mean,hbdist_std,hbnum_mean1,hbnum_std1,hbnum_mean2,hbnum_std2
43,3moj,2.857143,6.500905,13.814186,2.719199,18.559441,13.814186


# Compile into single file

In [27]:
X = df_features
Y = df_outputs
Y = Y[["PDB code 1","PDB code 2", "Output"]]
Y["PDB codes"] = Y["PDB code 1"] +" , "+ Y["PDB code 2"]
Y = Y[["PDB codes","Output"]]
Y.set_index("PDB codes",inplace=True)
X["PDB codes"] = X["PDB code 1"] +" , "+ X["PDB code 2"]
X.set_index("PDB codes", inplace=True)
X.drop(["PDB code 1", "PDB code 2"],axis=1,inplace=True)
df, right = X.align(Y,join="right",axis = 0)
df.dropna(inplace=True)
Y = Y[Y.index.isin(df.index)]
df["Output"] = Y["Output"]
df.to_csv("features_outputs.csv")

/var/folders/1l/f_c0h5451133klkyry12ld9h0000gn/T/ipykernel_27950/1653482531.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y["PDB codes"] = Y["PDB code 1"] +" , "+ Y["PDB code 2"]
